# Dataviews with SDS: Introduction 

Dataviews allows data scientist to create and use table views which can be vary from simple like the ones in this notebook, to very elaborate. 

The dataset is from the Deschutes Brewery, with many fermentors which are the assets with the data of interest. 

This notebook shows the steps involved in creating and using Dataviews. 

## Imports 

In [1]:
# To run this notebook outside Academic JupyterHub: pip install requests-futures
# 
# For parallel HTTP requests
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession
import requests
# Pandas dataframe
import pandas as pd
import io, datetime, json
from random import randint

## Tenant and client credentials

### NOTE: this info should be abstracted when we deploy for a course (TBD)

In [2]:
# Tenant is OSIsoft Samples, use your own credentials
tenant_id = '4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93'
client_id = 'ff8220f7-6b7c-4477-b21e-8e2ca20649d4'  # input('client_id:')  # 
client_secret = 'tRiVPtWc6kgcxEw090Qi/7nwA+JfI4cLlaL34Edgx+M='  # input('client_secret:')  # 

## Get an the autorization token and define the http for upcoming requests

In [3]:
# Request a fresh authorization bearer token 
authorization = requests.post('https://login.microsoftonline.com/%s/oauth2/token' % tenant_id,
                              data={'grant_type': 'client_credentials',
                                    'client_id': client_id,
                                    'client_secret': client_secret,
                                    'resource': 'https://pihomemain.onmicrosoft.com/ocsapi'
                                   })

In [4]:
# Required headers for SDS endpoint
headers = {'Authorization': 'bearer %s' % authorization.json()['access_token'],
           'Content-type': 'application/json',
           'Accept': 'text/plain', 
           'Request-Timeout': '60000' }

## Endpoint on MAIN cluster, namespace Brewing (data from Deschutes)

In [5]:
# Endpoint for dataview access
namespace_url = 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing'
endpoint = namespace_url + '/dataviews/'

## All data streams of a fermentor, here Fermentor 31
### (16 in total, listed in alphabetical order)

In [6]:
r = requests.get(namespace_url + '/Streams?query=Fermentor 31*', headers=headers)
if r.status_code == 200:
    streams = [stream for stream in r.json() if '31' in stream['Id']]
    for i, stream in enumerate(sorted(streams, key=lambda x: x['Id']), 1):
        print('%2d' % i, ':', stream['Id'], ' ((( type:', stream['TypeId'], ')))', '\n     Description:', stream['Description'], )
    print(r.json()[0])
else:
    print('Status:', r.status_code)

ConnectionError: HTTPSConnectionPool(host='historianmain.osipi.com', port=443): Max retries exceeded with url: /api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/Streams?query=Fermentor%2031* (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f0982b82e80>: Failed to establish a new connection: [Errno 110] Connection timed out',))

## Generate JSON for a Dataview

**TODO**: Documentation for dataview available at: TBD 

#### Function `generate_dataview_def` returns a pair: Dataview Id and Dataview JSON body definition.

#### The Dataview is built to return: 
* all data of Fermentor #`ferm_id` as rows
* each stream of the fermentor becomes a column _without the_ `Fermentor 3x` _prefix_ (important for dataframe concatenation)
* with data starting on UTC time `2017-03-18T00:00:00Z` up to 2017-04-08T00:00:00Z (20 days)
* with data interpolated at 1 minute (00:01:00) interval

In [7]:
# After changes on 11/27/2018
# Now mandatory: "IndexDataType": "DateTime"
version = randint(0, 99)
def generate_dataview_def(ferm_id):
   dataview_id = 'fermenter%d_20days_1min_v%d' % (ferm_id, version)
   return dataview_id, {
       "Id": dataview_id,
        "Name": dataview_id,
        "Queries": [
            {
                "Id" : "Fermentor",
                "Query" : {
                        "Type": "StreamName",
                        "Value": "Fermentor %d" % ferm_id,
                        "Operator": "Contains"
                }
            }
        ],
        "IndexDataType": "DateTime",
        "IndexConfig": {
            "IsDefault": False,
            "StartIndex": "2017-03-18T00:00:00Z",
            "EndIndex": "2017-04-08T00:00:00Z",
            "Mode": "Interpolated",
            "Interval": "00:01:00"
        }
   }

## Creation of the Dataviews, for fermenters 31 up to 36

* Status 201 from POST request indicates success
* Status 401 indicates unauthorized (try refreshing authorization header)
* Status 409 when a Dataview with same Id already exists (go to last cell of this notebook to perform a clean up)

In [8]:
dataviews = []
for ferm_id in range(31, 37):
    dataview_name, dataview_def = generate_dataview_def(ferm_id)
    dataviews.append(dataview_name)
    response = requests.post(endpoint + dataview_name, headers=headers, json=dataview_def)
    print('Status:', response.status_code, dataview_name, response.text[:80]+'...', )

Status: 201 fermenter31_20days_1min_v34 {"Id":"fermenter31_20days_1min_v34","Name":"fermenter31_20days_1min_v34","Querie...
Status: 201 fermenter32_20days_1min_v34 {"Id":"fermenter32_20days_1min_v34","Name":"fermenter32_20days_1min_v34","Querie...
Status: 201 fermenter33_20days_1min_v34 {"Id":"fermenter33_20days_1min_v34","Name":"fermenter33_20days_1min_v34","Querie...
Status: 201 fermenter34_20days_1min_v34 {"Id":"fermenter34_20days_1min_v34","Name":"fermenter34_20days_1min_v34","Querie...
Status: 201 fermenter35_20days_1min_v34 {"Id":"fermenter35_20days_1min_v34","Name":"fermenter35_20days_1min_v34","Querie...
Status: 201 fermenter36_20days_1min_v34 {"Id":"fermenter36_20days_1min_v34","Name":"fermenter36_20days_1min_v34","Querie...


## List of Dataviews URLs 

In [9]:
# We want 20 days of data worth at 1 minute interval, for fermenter 31 up to 36
dataviews_url = [endpoint + '%s/preview/interpolated?form=csvh' % d for d in dataviews]
dataviews_url

['https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter31_20days_1min_v34/preview/interpolated?form=csvh',
 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter32_20days_1min_v34/preview/interpolated?form=csvh',
 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter33_20days_1min_v34/preview/interpolated?form=csvh',
 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter34_20days_1min_v34/preview/interpolated?form=csvh',
 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter35_20days_1min_v34/preview/interpolated?form=csvh',
 'https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter36_20days_1min

## From a list of dataviews, gather them in parallel and return a single dataframe

In [10]:
# Request in parallel all the dataviews, return the concatenated dataframe
def get_ocs_dataframe(dataviews, headers, workers=8):
    ti = datetime.datetime.now()
    session = FuturesSession(executor=ThreadPoolExecutor(max_workers=workers))
    rs = [session.get(u, headers=headers) for u in dataviews]
    resps = [r.result() for r in rs]
    print('Requests completed in', datetime.datetime.now() - ti) 
    print(resps)
    dfs = [pd.read_csv(io.StringIO(r.text), parse_dates=['_time']) for r in resps]
    return(pd.concat(dfs, sort=True))

## Get dataframe and time it (about 20 seconds, be patient)

All responses should HTTP code [200] if everything is ok 

In [11]:
t0 = datetime.datetime.now()
df = get_ocs_dataframe(dataviews_url, headers)
print('Dataframe obtained in', datetime.datetime.now() - t0) 
df.info

Requests completed in 0:00:20.741605
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
Dataframe obtained in 0:00:21.727244


<bound method DataFrame.info of             ADF  Bottom_TIC_OUT  Bottom_TIC_PV  Bottom_Temperature     Brand  \
0      0.659725       15.586471      30.011260                   0  Kerberos   
1      0.659725       15.586471      30.011260                   0  Kerberos   
2      0.659725       15.586471      30.011260                   0  Kerberos   
3      0.659725       15.586471      30.011260                   0  Kerberos   
4      0.659725       15.586471      30.011260                   0  Kerberos   
5      0.659725       15.586471      30.011260                   0  Kerberos   
6      0.659725       15.586471      30.011260                   0  Kerberos   
7      0.659725       15.586471      30.011260                   0  Kerberos   
8      0.659725       15.586471      30.011260                   0  Kerberos   
9      0.659725       15.586471      30.011260                   0  Kerberos   
10     0.659725       15.586471      30.011260                   0  Kerberos   
11     0

### Note the that resulting dataframe has almost 182K rows

## Save data into CSV file locally in directory FlashcARD

In [12]:
df.to_csv('beer_20_days.csv')

---
## Clicking this [link](./beer_20_days.csv) opens up a CSV browser with the CSV above
---

### List of column names with their type

Note that the `_time` column (a new column on top of the 16 ones of a Fermentor) has the correct datetime panda data type for timestamps

In [13]:
for i, c in enumerate(df.columns, 1):
    print('%2d' % i, c, '((( type:', df[c].dtype, ')))')

 1 ADF ((( type: float64 )))
 2 Bottom_TIC_OUT ((( type: float64 )))
 3 Bottom_TIC_PV ((( type: float64 )))
 4 Bottom_Temperature ((( type: int64 )))
 5 Brand ((( type: object )))
 6 DefaultGroupRule ((( type: object )))
 7 FV_Full_Plato ((( type: float64 )))
 8 Fermentation_ID ((( type: object )))
 9 Middle_TIC_OUT ((( type: float64 )))
10 Middle_TIC_PV ((( type: float64 )))
11 Plato ((( type: float64 )))
12 Quality ((( type: float64 )))
13 Status ((( type: object )))
14 Top_TIC_OUT ((( type: float64 )))
15 Top_TIC_PV ((( type: float64 )))
16 Volume ((( type: float64 )))
17 _time ((( type: datetime64[ns] )))


### List of unique Fermentation ID

We want to track the Apparent Degree of Fermentation (ADF) per fermentation batch tracked by ID 

In [14]:
for f in df.Fermentation_ID.unique():
    print(f, isinstance(f, str))

Fermentor 31201731179653 True
nan False
Fermentor 31201732314856 True
FV322016113055113 True
Fermentor 33201731511870 True
Fermentor 3320173183371 True
FV342016112860676 True
FV35201612449149 True
Fermentor 36201731679561 True


### Prepare ADF curve plots over time 

In [15]:
import plotly.graph_objs as go

figs = []
data = []
for f in df.Fermentation_ID.unique():
    trace = go.Scattergl(x = df[df.Fermentation_ID == f]['_time'], y = df[df.Fermentation_ID == f]['ADF'], mode='lines+markers', name=str(f))
    figs.append(go.FigureWidget(data=[trace]))
    data.append(trace)

### Add a range slider 

With a few time range selectors: 8 hours, 1 day and everything 

Note: range slider is grey now because of an incompatibility with ScatterGL: https://github.com/plotly/plotly.js/issues/2627

In [16]:
layout = dict(
    title='Brewing ADF with time range slider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=8,
                     label='8h',
                     step='hour',
                     stepmode='backward'),
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)
        
fig = go.FigureWidget(data=data, layout=layout)
fig

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Fermentor 31201731179653',
      …

## Clean up: delete Dataviews  

* Code 204 if deletion is successful
* Code 404 if requested Dataview Id doesn't exist or already deleted

In [17]:
for dv in dataviews_url:
    dv_url = dv[:dv.find('/preview')]
    s = requests.delete(dv_url, headers=headers)
    print(s.status_code, dv_url)

204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter31_20days_1min_v46
204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter32_20days_1min_v46
204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter33_20days_1min_v46
204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter34_20days_1min_v46
204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter35_20days_1min_v46
204 https://historianmain.osipi.com/api/Tenants/4fa85df4-9f5a-49f8-954f-dcf0d6e1ff93/Namespaces/Brewing/dataviews/fermenter36_20days_1min_v46
